In [1]:
### http://karthur.org/2015/clipping-rasters-in-python.html

In [2]:
from osgeo import gdal, gdalnumeric, ogr
from PIL import Image, ImageDraw
import os, sys
import numpy as np

def clip_raster(rast, features_path, gt=None, nodata=-9999):
    '''
    Clips a raster (given as either a gdal.Dataset or as a numpy.array
    instance) to a polygon layer provided by a Shapefile (or other vector
    layer). If a numpy.array is given, a "GeoTransform" must be provided
    (via dataset.GetGeoTransform() in GDAL). Returns an array. Clip features
    must be a dissolved, single-part geometry (not multi-part). Modified from:

    http://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html
    #clip-a-geotiff-with-shapefile

    Arguments:
        rast            A gdal.Dataset or a NumPy array
        features_path   The path to the clipping features
        gt              An optional GDAL GeoTransform to use instead
        nodata          The NoData value; defaults to -9999.
    '''
    def array_to_image(a):
        '''
        Converts a gdalnumeric array to a Python Imaging Library (PIL) Image.
        '''
        i = Image.fromstring('L',(a.shape[1], a.shape[0]),
            (a.astype('b')).tostring())
        return i

    def image_to_array(i):
        '''
        Converts a Python Imaging Library (PIL) array to a gdalnumeric image.
        '''
        a = gdalnumeric.fromstring(i.tobytes(), 'b')
        a.shape = i.im.size[1], i.im.size[0]
        return a

    def world_to_pixel(geo_matrix, x, y):
        '''
        Uses a gdal geomatrix (gdal.GetGeoTransform()) to calculate
        the pixel location of a geospatial coordinate; from:
        http://pcjericks.github.io/py-gdalogr-cookbook/raster_layers.html#clip-a-geotiff-with-shapefile
        '''
        ulX = geo_matrix[0]
        ulY = geo_matrix[3]
        xDist = geo_matrix[1]
        yDist = geo_matrix[5]
        rtnX = geo_matrix[2]
        rtnY = geo_matrix[4]
        pixel = int((x - ulX) / xDist)
        line = int((ulY - y) / xDist)
        return (pixel, line)

    # Can accept either a gdal.Dataset or numpy.array instance
    if not isinstance(rast, np.ndarray):
        gt = rast.GetGeoTransform()
        rast = rast.ReadAsArray()

    # Create an OGR layer from a boundary shapefile
    features = ogr.Open(features_path)
    if features.GetDriver().GetName() == 'ESRI Shapefile':
        lyr = features.GetLayer(os.path.split(os.path.splitext(features_path)[0])[1])

    else:
        lyr = features.GetLayer()

    # Get the first feature
    poly = lyr.GetNextFeature()

    # Convert the layer extent to image pixel coordinates
    minX, maxX, minY, maxY = lyr.GetExtent()
    ulX, ulY = world_to_pixel(gt, minX, maxY)
    lrX, lrY = world_to_pixel(gt, maxX, minY)

    # Calculate the pixel size of the new image
    pxWidth = int(lrX - ulX)
    pxHeight = int(lrY - ulY)

    # If the clipping features extend out-of-bounds and ABOVE the raster...
    if gt[3] < maxY:
        # In such a case... ulY ends up being negative--can't have that!
        iY = ulY
        ulY = 0

    # Multi-band image?
    try:
        clip = rast[:, ulY:lrY, ulX:lrX]

    except IndexError:
        clip = rast[ulY:lrY, ulX:lrX]

    # Create a new geomatrix for the image
    gt2 = list(gt)
    gt2[0] = minX
    gt2[3] = maxY

    # Map points to pixels for drawing the boundary on a blank 8-bit,
    #   black and white, mask image.
    points = []
    pixels = []
    geom = poly.GetGeometryRef()
    pts = geom.GetGeometryRef(0)

    for p in range(pts.GetPointCount()):
        points.append((pts.GetX(p), pts.GetY(p)))

    for p in points:
        pixels.append(world_to_pixel(gt2, p[0], p[1]))

    raster_poly = Image.new('L', (pxWidth, pxHeight), 1)
    rasterize = ImageDraw.Draw(raster_poly)
    rasterize.polygon(pixels, 0) # Fill with zeroes

    # If the clipping features extend out-of-bounds and ABOVE the raster...
    if gt[3] < maxY:
        # The clip features were "pushed down" to match the bounds of the
        #   raster; this step "pulls" them back up
        premask = image_to_array(raster_poly)
        # We slice out the piece of our clip features that are "off the map"
        mask = np.ndarray((premask.shape[-2] - abs(iY), premask.shape[-1]), premask.dtype)
        mask[:] = premask[abs(iY):, :]
        mask.resize(premask.shape) # Then fill in from the bottom

        # Most importantly, push the clipped piece down
        gt2[3] = maxY - (maxY - gt[3])

    else:
        mask = image_to_array(raster_poly)

    # Clip the image using the mask
    try:
        clip = gdalnumeric.choose(mask, (clip, nodata))

    # If the clipping features extend out-of-bounds and BELOW the raster...
    except ValueError:
        # We have to cut the clipping features to the raster!
        rshp = list(mask.shape)
        if mask.shape[-2] != clip.shape[-2]:
            rshp[0] = clip.shape[-2]

        if mask.shape[-1] != clip.shape[-1]:
            rshp[1] = clip.shape[-1]

        mask.resize(*rshp, refcheck=False)

        clip = gdalnumeric.choose(mask, (clip, nodata))

    return (clip, ulX, ulY, gt2)


In [4]:
#raster_path = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/oh/oh.tif'
raster_path = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/256x256/ortho_1.tif'
#'/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/ortho/2018250_bischofshofen_lb_rgbi.img'
shp_path = '/media/philipp/ed7d22ba-5a3b-4d31-bf6c-6add6e106b3d/test/rater_clip/test_256.shp'

In [5]:
# Load the source data as a gdalnumeric array
#srcArray = gdalnumeric.LoadFile(raster_path)

# Also load as a gdal image to get geotransform
# (world file) info
srcImage = gdal.Open(raster_path)

In [6]:
clip, ulX, ulY, gt2 = clip_raster(srcImage, shp_path, gt=None, nodata=-9999)

<ipython-input-2-04f6c83ce52d>:35: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead
  a = gdalnumeric.fromstring(i.tobytes(), 'b')


In [7]:
#gdalnumeric.SaveArray(clip, "OUTPUT.tif", format="GTiff", prototype=raster_path)

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x7ffa9d67eb40> >

In [10]:
clip.shape

(4, 260, 260)

In [15]:
clip.shape[1]

260

In [8]:
dst_filename = 'xxx_oh.tiff'
x_pixels = clip.shape[0]  # number of pixels in x
y_pixels = clip.shape[1]  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename,x_pixels, y_pixels, 1,gdal.GDT_Float32)
dataset.GetRasterBand(1).WriteArray(clip)

# follow code is adding GeoTranform and Projection
#geotrans=srcImage.GetGeoTransform()  #get GeoTranform from existed 'data0'
proj=srcImage.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(gt2)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

In [16]:
dst_filename = 'xxx_ortho.tiff'
x_pixels = clip.shape[1]  # number of pixels in x
y_pixels = clip.shape[2]  # number of pixels in y
driver = gdal.GetDriverByName('GTiff')
dataset = driver.Create(dst_filename, x_pixels, y_pixels, clip.shape[0], gdal.GDT_Byte)
for i in range(clip.shape[0]):
    dataset.GetRasterBand(i+1).WriteArray(clip[i])

# follow code is adding GeoTranform and Projection
#geotrans=srcImage.GetGeoTransform()  #get GeoTranform from existed 'data0'
proj=srcImage.GetProjection() #you can get from a exsited tif or import 
dataset.SetGeoTransform(gt2)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

In [18]:
clip

array([[[ 54,  54,  51, ...,  66,  76,  84],
        [ 61,  63,  68, ...,  50,  49,  92],
        [ 66,  72,  68, ...,  44,  45,  42],
        ...,
        [ 69,  91,  81, ...,  77,  74,  72],
        [ 64,  64,  71, ..., 105, 115, 111],
        [ 68,  64,  70, ..., 117, 102, 110]],

       [[ 58,  57,  56, ...,  70,  84,  90],
        [ 64,  64,  70, ...,  53,  49, 104],
        [ 68,  77,  73, ...,  49,  48,  47],
        ...,
        [ 72,  95,  80, ...,  76,  75,  73],
        [ 68,  68,  73, ..., 108, 119, 115],
        [ 74,  69,  75, ..., 130, 103, 118]],

       [[ 57,  56,  54, ...,  62,  69,  69],
        [ 60,  58,  66, ...,  50,  44,  82],
        [ 58,  67,  64, ...,  48,  47,  42],
        ...,
        [ 61,  76,  61, ...,  62,  57,  55],
        [ 60,  58,  56, ...,  78,  91,  80],
        [ 65,  61,  62, ..., 100,  80,  92]],

       [[124, 120, 112, ..., 153, 176, 184],
        [153, 150, 156, ..., 116, 127, 188],
        [166, 172, 167, ...,  80,  87,  96],
        ..